In [1]:
import numpy as np
import keras
from keras import backend as K

Using TensorFlow backend.


In [2]:
from keras.initializers import random_uniform

from keras.layers import Layer, Input, Conv1D, Dense

from keras.models import Model, Sequential

In [3]:
import wFmConv

In [4]:
from imp import reload

In [ ]:
reload(wFmConv)

In [145]:
def point_tReLU(z):
    
    if 'complex' not in str(type(z)): 
        r, theta = wFmConv.identify(z[0], z[1])
    else:
        r, theta = wFmConv.identify(np.real(z), np.imag(z))
    r_hat = K.max(K.tf.convert_to_tensor([r,1]))
    theta_hat = K.max([theta,0])
    re = r_hat*K.tf.cos(theta_hat)
    im = r_hat*K.tf.sin(theta_hat)
    return [re, im]

In [146]:
def tReLU(complex_array):
    # To standardize, let's make the array _ x 2.
    if 'complex' in str(type(complex_array[0])):
        new_array = []
        for cnum in complex_array:
            new = [np.real(cnum), np.imag(cnum)]
            new_array.append(new)
        complex_array = np.array(new_array)
    # Iterate through the complex numbers and compute tReLU of each.
    output = []
    for ind in range(complex_array.shape[0]):
        out = point_tReLU(complex_array[ind])
        output.append(out)
    return K.tf.convert_to_tensor(output)
    

In [147]:
def get_output_shape_trelu(input_shape):
    return input_shape

In [148]:
def invariantLayer(complex_array, weights):
    t_u = wFmConv.calc_wfm(complex_array, weights) # [real, imag]
    output = []
    for i in range(len(weights)):
        out = wFmConv.get_manifold_distance(complex_array[i], t_u)
        output.append(out)
    output = np.array(output)
    return K.tf.convert_to_tensor(output)  

In [149]:
class complexConv(Layer):
    def __init__(self, out_length, kernel_size=3, stride=1, \
                 de_crossover_prob=0.95, de_diffwt=1.0, \
                 de_num_iters=50, verbose=0):
        self.out_length = out_length
        self.kernel_size = kernel_size
        self.stride = stride
        self.de_crossover_prob = de_crossover_prob
        self.de_diffwt = de_diffwt
        self.de_num_iters = de_num_iters
        self.verbose = verbose
        super(complexConv, self).__init__()
        
    # initial input shape: (None, 128, 2)
    def build(self, input_shape):
        # Trainable weight variable
        self.kernel = self.add_weight(name='kernel', \
                                    shape=(input_shape[1], 1), \
                                    initializer=K.random_uniform, \
                                    trainable=True)
        
        super(complexConv, self).build(input_shape)
    
    def call(self, x):
        output = []
        for i in range(x.shape[0]):
            output_i = []
            for j in range(self.out_length):
                start = j*self.stride
                datachunk = x[i][start:start+self.kernel_size]
                datachunk = K.cast(datachunk, 'float32')
                weightlist = self.kernel[i][start:start+self.kernel_size]
                weightlist = K.cast(weightlist, 'float32')
                out = wFmConv.calc_wfm(datachunk, weightlist, \
                    num_iters=self.de_num_iters, \
                    crossover_prob=self.de_crossover_prob, \
                    F=self.de_diffwt, verbose=self.verbose)
                output_i.append(out)
            output.append(output_i)
        return K.tf.convert_to_tensor(output)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.out_length, input_shape[-1])

In [150]:
numSamples = 10

In [151]:
in_shape = (2,numSamples,2)

In [152]:
in_layer = Input(batch_shape=in_shape)

In [153]:
in_layer

<tf.Tensor 'input_7:0' shape=(2, 10, 2) dtype=float32>

In [154]:
cc1 = complexConv(8, de_num_iters=2)(in_layer)

(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)


In [155]:
cc1

<tf.Tensor 'complex_conv_9/packed:0' shape=(2, 8, 2) dtype=float32>

In [156]:
from keras.layers import Lambda

In [157]:
m = Model([in_layer], [cc1])

In [158]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (2, 10, 2)                0         
_________________________________________________________________
complex_conv_9 (complexConv) (2, 8, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [159]:
example_input = np.random.randn(2,10,2)

In [160]:
m.output_shape

(2, 8, 2)

In [161]:
m.assert_input_compatibility(K.tf.convert_to_tensor(example_input))

In [162]:
m.predict(example_input, steps=1)

array([[[ 0.2603027 ,  0.5929388 ],
        [-0.02943194,  0.46154547],
        [ 0.62689894,  0.14877038],
        [ 0.5380855 ,  0.20685126],
        [ 0.6973565 , -0.0305621 ],
        [ 0.5269194 ,  0.51472634],
        [ 1.3229812 ,  0.06138201],
        [ 0.6663172 ,  0.73901635]],

       [[-0.15949757,  0.5959659 ],
        [-0.09599105,  0.51658154],
        [ 0.86901623,  0.33479002],
        [ 1.3488173 , -0.24277003],
        [ 1.2669624 , -0.2611048 ],
        [ 0.58823746, -0.26739845],
        [-0.74967545, -0.49357498],
        [-0.235403  , -0.5931781 ]]], dtype=float32)

In [163]:
trelu1 = Lambda(tReLU, output_shape=get_output_shape_trelu)(cc1)

ValueError: Shapes must be equal rank, but are 1 and 0
	From merging shape 0 with other shapes. for 'lambda_7/packed' (op: 'Pack') with input shapes: [2], [].

In [ ]:
trelu1

In [92]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (1, 10, 2)                0         
_________________________________________________________________
complex_conv_5 (complexConv) (1, 8, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cc2 = complexConv(6, de_num_iters=2)(trelu1)